In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import moscot.plotting as mpl
from moscot.problems.cross_modality import TranslationProblem
from moscot.problems.generic import GWProblem

import scanpy as sc
import anndata as ad
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing as pp

import time
from tqdm import tqdm
import seaborn as sns
import scib_metrics
import scib
import helper_functions as fcts

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)

In [3]:
import pygsp as pygsp

In [5]:
def align(i, adata_src, emb_src, adata_tgt, emb_tgt, emb_joint, 
          cust_cost_xy, cust_cost_x, cust_cost_y, 
          epsilon, alpha,
          geo_cost_linear = False, geo_cost_quad = False):
    
    adata = sc.concat([adata_src, adata_tgt], join = 'outer', label='batch', index_unique = '-')
    tp = GWProblem(adata)
    tp = tp.prepare(
        key="batch", 
        x_attr={"attr": "obsm", "key": emb_src}, 
        y_attr={"attr": "obsm", "key": emb_tgt}, 
        joint_attr=emb_joint
    )
    
    if geo_cost_linear:
        tp[('0', '1')].set_xy(cust_cost_xy, tag="cost_matrix")
    if geo_cost_quad:
        tp[('0', '1')].set_x(cust_cost_x, tag="cost_matrix")  
        tp[('0', '1')].set_y(cust_cost_y, tag="cost_matrix") 
        
    tp = tp.solve(alpha=alpha, epsilon=epsilon)
    
    conv = tp[('0', '1')].solution.converged
    T = tp[('0', '1')].solution.transport_matrix
    translated_norm = T.dot(adata_tgt.obsm[emb_tgt])/ np.sum(T,axis=1)[:,None]
    
    adata.obsm["X_fgw"] = np.concatenate((translated_norm, adata_tgt.obsm[emb_tgt]), axis=0)
    
    # Evaluation metrics
    f = fcts.foscttm(translated_norm, adata_tgt.obsm[emb_tgt])
    lta = fcts.transfer_accuracy(translated_norm, adata_tgt.obsm[emb_tgt], adata_src.obs['cell_type'], adata_tgt.obs['cell_type'])
    alg_score = fcts.alignment_score(np.array(translated_norm), np.array(adata_tgt.obsm[emb_tgt]))
    rmse = fcts.calculate_rmse(translated_norm, adata_tgt.obsm[emb_tgt])
    
    # Bio-conservation metrics
    sc.pp.neighbors(adata, use_rep='X_fgw')
    scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
    nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
    asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_fgw')
    ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')
    # Batch correction metrics
    asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_fgw',verbose=False)
    graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')
    
    return [i, epsilon, alpha, emb_src, emb_tgt, geo_cost_linear, geo_cost_quad, conv, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity]

In [14]:
# Custom Costs
sc.pp.neighbors(geneactivity, use_rep ="X_scVI", key_added="multi_vi", n_neighbors=30)
cell_ids_source = geneactivity[geneactivity.obs['batch']=='0'].obs_names
cell_ids_target = geneactivity[geneactivity.obs['batch']=='1'].obs_names
G = pygsp.graphs.Graph(geneactivity.obsp["multi_vi_connectivities"])
G.estimate_lmax()
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(geneactivity)))
cc_xy = pd.DataFrame(data=-np.log(diffusion_distances[:len(cell_ids_source), len(cell_ids_source):].copy())+1e-15, index=cell_ids_source, columns=cell_ids_target)

sc.pp.neighbors(rna, use_rep = 'X_pca', key_added = "multi_vi", n_neighbors = 30)
G = pygsp.graphs.Graph(rna.obsp["multi_vi_connectivities"])
G.estimate_lmax()
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(rna)))
cell_ids_src = adata[adata.obs['batch']=='0'].obs_names
cc_x_pca = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_src, columns=cell_ids_src)

sc.pp.neighbors(atac, use_rep = 'X_lsi_red', key_added = "multi_vi", n_neighbors = 30)
G = pygsp.graphs.Graph(atac.obsp["multi_vi_connectivities"])
G.estimate_lmax()
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(atac)))
cell_ids_tgt = adata[adata.obs['batch']=='1'].obs_names
cc_y_lsi = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_tgt, columns=cell_ids_tgt)

/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'adata' is not defined

In [16]:
sc.pp.neighbors(rna, use_rep = 'X_pca', key_added = "multi_vi", n_neighbors = 30)
G = pygsp.graphs.Graph(rna.obsp["multi_vi_connectivities"])
G.estimate_lmax()
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(rna)))
cell_ids_src = adata[adata.obs['batch']=='0'].obs_names
cc_x_pca = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_src, columns=cell_ids_src)

sc.pp.neighbors(atac, use_rep = 'X_lsi_red', key_added = "multi_vi", n_neighbors = 30)
G = pygsp.graphs.Graph(atac.obsp["multi_vi_connectivities"])
G.estimate_lmax()
filt = pygsp.filters.Heat(G, tau=100)
diffusion_distances = filt.filter(np.eye(len(atac)))
cell_ids_tgt = adata[adata.obs['batch']=='1'].obs_names
cc_y_lsi = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_tgt, columns=cell_ids_tgt)

# Grid search

In [18]:
for i in tqdm(['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3', 's3d6', 's3d7', 's4d1', 's4d8', 's4d9']):
    atac = sc.read('data/'+i+'/atac-emb_gc.h5ad')
    rna = sc.read('data/'+i+'/rna-emb_gc.h5ad')
    geneactivity = sc.read('data/'+i+'/gene_activity_scvi.h5ad')
    rna.obsm['X_pca_norm'] = pp.normalize(rna.obsm['X_pca'])
    atac.obsm['X_lsi_norm'] = pp.normalize(atac.obsm['X_lsi_red']) 
    rna.obsm['X_scVI_norm'] = pp.normalize(rna.obsm['X_scVI'])
    atac.obsm['X_poissonVI_norm'] = pp.normalize(atac.obsm['X_poissonVI'])
    rna.obsm['geneactivity'] = geneactivity[geneactivity.obs['batch']=='0'].obsm['X_scVI']
    atac.obsm['geneactivity'] = geneactivity[geneactivity.obs['batch']=='1'].obsm['X_scVI']
    adata = ad.concat([rna, atac], join = 'outer', label='batch', index_unique = '-')

    # Custom Costs
    sc.pp.neighbors(geneactivity, use_rep ="X_scVI", key_added="multi_vi", n_neighbors=30)
    cell_ids_source = geneactivity[geneactivity.obs['batch']=='0'].obs_names
    cell_ids_target = geneactivity[geneactivity.obs['batch']=='1'].obs_names
    G = pygsp.graphs.Graph(geneactivity.obsp["multi_vi_connectivities"])
    G.estimate_lmax()
    filt = pygsp.filters.Heat(G, tau=100)
    diffusion_distances = filt.filter(np.eye(len(geneactivity)))
    cc_xy = pd.DataFrame(data=-np.log(diffusion_distances[:len(cell_ids_source), len(cell_ids_source):].copy())+1e-15, index=cell_ids_source, columns=cell_ids_target)

    sc.pp.neighbors(rna, use_rep = 'X_pca', key_added = "multi_vi", n_neighbors = 30)
    G = pygsp.graphs.Graph(rna.obsp["multi_vi_connectivities"])
    G.estimate_lmax()
    filt = pygsp.filters.Heat(G, tau=100)
    diffusion_distances = filt.filter(np.eye(len(rna)))
    cell_ids_src = adata[adata.obs['batch']=='0'].obs_names
    cc_x_pca = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_src, columns=cell_ids_src)

    sc.pp.neighbors(atac, use_rep = 'X_lsi_red', key_added = "multi_vi", n_neighbors = 30)
    G = pygsp.graphs.Graph(atac.obsp["multi_vi_connectivities"])
    G.estimate_lmax()
    filt = pygsp.filters.Heat(G, tau=100)
    diffusion_distances = filt.filter(np.eye(len(atac)))
    cell_ids_tgt = adata[adata.obs['batch']=='1'].obs_names
    cc_y_lsi = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_tgt, columns=cell_ids_tgt)

    sc.pp.neighbors(rna, use_rep = 'X_scVI', key_added = "multi_vi", n_neighbors = 30)
    G = pygsp.graphs.Graph(rna.obsp["multi_vi_connectivities"])
    G.estimate_lmax()
    filt = pygsp.filters.Heat(G, tau=100)
    diffusion_distances = filt.filter(np.eye(len(rna)))
    cell_ids_src = adata[adata.obs['batch']=='0'].obs_names
    cc_x_scvi = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_src, columns=cell_ids_src)

    sc.pp.neighbors(atac, use_rep = 'X_poissonVI', key_added = "multi_vi", n_neighbors = 30)
    G = pygsp.graphs.Graph(atac.obsp["multi_vi_connectivities"])
    G.estimate_lmax()
    filt = pygsp.filters.Heat(G, tau=100)
    diffusion_distances = filt.filter(np.eye(len(atac)))
    cell_ids_tgt = adata[adata.obs['batch']=='1'].obs_names
    cc_y_poisson = pd.DataFrame(data=-np.log(diffusion_distances.copy())+1e-15, index=cell_ids_tgt, columns=cell_ids_tgt)

    for epsilon in tqdm([1e-3,.5e-2,1e-2,.5e-1,1e-1, .5]):
        for alpha in tqdm([0.25, 0.5, 0.75, 0.9, 0.95, 0.98]):
            for src_emb, tgt_emb in [('X_pca_norm', 'X_lsi_norm'), ('X_scVI_norm', 'X_poissonVI_norm')]:
                if src_emb == 'X_pca_norm':
                    cc_x = cc_x_pca
                    cc_y = cc_y_lsi
                else:
                    cc_x = cc_x_scvi
                    cc_y = cc_y_poisson
                for geo_cost_linear in [True, False]:
                    for geo_cost_quad in [True, False]:
                        out = align(i, adata_src = rna, emb_src = src_emb, 
                                  adata_tgt = atac, emb_tgt = tgt_emb,
                                  emb_joint = 'geneactivity',
                                  cust_cost_xy = cc_xy, cust_cost_x = cc_x, cust_cost_y = cc_y,
                                  epsilon = epsilon, alpha = alpha,
                                  geo_cost_linear = geo_cost_linear, geo_cost_quad = geo_cost_quad)
                        results.append(out)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4279, 4279)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)


WARNING  Solver did not converge                                                                                   


/tmp/ipykernel_2183343/458423687.py:37: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


...

/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2183343/458423687.py:37: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4279, 4279)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2183343/458423687.py:37: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4279, 4279)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2183343/458423687.py:37: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)


100%|██████████| 6/6 [23:43<00:00, 237.24s/it]

100%|██████████| 1/1 [2:43:11<00:00, 9791.14s/it]


In [19]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "epsilon",'alpha', 'emb_src', 'emb_tgt', 'geo_cost_linear', 'geo_cost_quad', "converged", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)
df_results
df_results.to_csv('results_fgw.csv', index = False)